In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images
x_train = x_train.reshape((-1, 28*28))
x_test = x_test.reshape((-1, 28*28))

# Hyperparameters
batch_sizes = [1, 10, 100]
epochs_list = [10, 50, 100]
learning_rate = 0.1
hidden_units = 256

# To store results
results = []

# Loop through all combinations
for batch_size in batch_sizes:
    for epochs in epochs_list:
        print(f"Training with batch size = {batch_size}, epochs = {epochs}")

        # Build model
        model = models.Sequential([
            layers.Input(shape=(784,)),
            layers.Dense(hidden_units, activation='relu'),
            layers.Dense(hidden_units, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])

        optimizer = optimizers.SGD(learning_rate=learning_rate)
        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Time tracking
        start_time = time.time()

        # Train
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=0,
                            validation_data=(x_test, y_test))

        end_time = time.time()
        duration = end_time - start_time

        # Evaluate
        y_pred = model.predict(x_test, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)
        acc = accuracy_score(y_test, y_pred_classes)
        cm = confusion_matrix(y_test, y_pred_classes)

        print(f"Accuracy: {acc*100:.2f}% | Time Taken: {duration:.2f} seconds")

        # Save plots
        suffix = f"bs{batch_size}_ep{epochs}"

        # Accuracy & Loss Curve
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Acc')
        plt.plot(history.history['val_accuracy'], label='Val Acc')
        plt.title(f'Accuracy Curve ({suffix})')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f'Loss Curve ({suffix})')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"accuracy_loss_{suffix}.png")
        plt.close()

        # Confusion Matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
        plt.title(f'Confusion Matrix ({suffix})')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(f"confusion_matrix_{suffix}.png")
        plt.close()

        # Append results
        results.append({
            'Batch Size': batch_size,
            'Epochs': epochs,
            'Accuracy': round(acc * 100, 2),
            'Time (s)': round(duration, 2),
            'Final Loss': round(history.history['val_loss'][-1], 4),
            'Final Accuracy': round(history.history['val_accuracy'][-1], 4)
        })

# Save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("experiment_results.csv", index=False)
print("\nAll experiment results saved to 'experiment_results.csv'")
